In [3]:
import math

In [84]:
# Givens
d = 7.94e-3  # [m]
L = ...  # [m]
sin_theta = 1 / 150  # []
h0, hf = 10e-2, 2e-2  # [m], [m]
L_box, W_box = 32e-2, 26e-2  # [m], [m]
eps = 0.0015e-3  # [m] (surface roughness of PVC pipe)
mu = 0.89e-3 #  [Pa * s] (water at 25 degrees C)

# Constants
rho = 997  # [kg/m^3]
g = 9.81  # [m/s^2]

# Initial Conditions
A_tube = math.pi * d ** 2 / 4  # [m^2]
A_box = L_box * W_box  # [m^2]
V0 = A_box * (h0 + hf)  # [m^3]

# Final Conditions
VF = A_box * hf

In [5]:
# Experimental Results
results = {
    20: (3, 19),
    30: (3, 34),
    40: (4, 26),
    60: (4, 48)
}
results = {Li * 1e-2: 60 * mins + secs for Li, (mins, secs) in results.items()}

def errors(model) -> int:
    for Li, t_avg in results.items():
        t_pred = model(Li)
        print(f"{Li=}m {t_avg=}s {t_pred=}s sqr_error={(t_avg - t_pred) ** 2}")

In [6]:
# Model 1: Simple Bernoulli's Equation
# Ignores: losses, height difference, tube length (!!)

# (1 / (2 * A_tube ** 2)) * (V') ** 2 =  (g / A_box) * V
k = A_tube * (2 * g / A_box) ** 0.5
# dV/dt = -k * sqrt(V)
# sqrt(V) = -kt + C
C = 2 * V0 ** 0.5

def V(t):
    """Volume of tank as a function of time"""
    return (-k / 2 * t + C/2) ** 2

# find root of function
# VF = (-k / 2 * t + V0 ** 0.5) ** 2
# t = 2/k (sqrt(V0) - sqrt(VF))

def m1_t(L):
    """Time to drain as a funciton of tube length"""
    return 2/k * (V0 ** 0.5 - VF ** 0.5)  # [s]

In [131]:
def reynolds_number(v):
    return rho * v * d / mu

def haaland_equation(re):
    inv_sqrt_f = -1.8 * math.log10((eps / d / 3.7) ** 1.1 + 6.9 / re)
    return (1 / inv_sqrt_f) ** 2

def s_t_min_s(s):
    return f"{s//60}:{(s%60):02}"

def solve(l):
    Vi = V0
    Vdot_i = 0
    n = 0
    delta_t = 0.00001  # [s]
    while True:
        n += 1
        Vi += Vdot_i * delta_t  # [m^3/s]
        vi = -Vdot_i / A_tube  # [m/s]

        if vi == 0:
            Vdot_i = -A_tube * (2 * g / A_box * Vi) ** 0.5
        else:
            re = reynolds_number(vi)
            if re > 4000:
                f = haaland_equation(re)
                Vdot_i = -A_tube * ((Vi / A_box) * (1 / (2 * g) * (l / d * f + 1)) ** -1) ** 0.5
            else:
                Vdot_i = -A_tube * (g / A_box * Vi) ** 0.5

        if abs((Vi - VF) / VF) < 0.001:
            print(l, "=>", s_t_min_s(int(n * delta_t)))
            break
        elif n > 500 / delta_t:
            print("too many steps")
            break

for l in [0.2, 0.3, 0.4, 0.6]:
    solve(l)

KeyboardInterrupt: 